In [1]:
import numpy as np
import sys
import qutip as qtp 
import matplotlib.pyplot as plt
from scipy import constants
import pysqkit
from pysqkit import Qubit
import xarray
from pysqkit.util.linalg import get_mat_elem, tensor_prod
from itertools import product
from typing import List
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
import json

In [2]:
with open('../../flx_transm_params.txt') as param_file:
    parameters_set = json.load(param_file)

In [3]:
#Transmon

p_set_cr = "CR_3"
levels_t = 3
transm = pysqkit.qubits.SimpleTransmon(
    label='T', 
    max_freq=parameters_set[p_set_cr]["max_freq_t"], 
    anharm=parameters_set[p_set_cr]["anharm_t"],
    diel_loss_tan=parameters_set[p_set_cr]["diel_loss_tan_t"], #set to zero to check d_1 L1 = d_2 L2
    dim_hilbert=levels_t,
    dephasing_times=parameters_set[p_set_cr]["dephasing_times_t"]
)

levels_f = 8

flx_cr = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set[p_set_cr]["charge_energy_f"], 
    induct_energy=parameters_set[p_set_cr]["induct_energy_f"], 
    joseph_energy=parameters_set[p_set_cr]["joseph_energy_f"], 
)
flx_cr.diagonalize_basis(levels_f)
flx_cphase = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set["CPHASE"]["charge_energy_f"], 
    induct_energy=parameters_set["CPHASE"]["induct_energy_f"], 
    joseph_energy=parameters_set["CPHASE"]["joseph_energy_f"], 
)
flx_cphase.diagonalize_basis(levels_f)

flx_dict = {"cr": flx_cr, "cphase": flx_cphase}

In [4]:
flx_transitions = [["0", "1"], ["1", "2"], ["0", "3"]]
transm_transitions = [["0", "1"], ["1", "2"]]

flx_data = {}
flx_data["cr"] = []
flx_data["cphase"] = []

for key in flx_dict.keys():
    for transition in flx_transitions:
        state_a = flx_dict[key].state(transition[0])
        state_b = flx_dict[key].state(transition[1])
        freq = np.abs(state_a[0] - state_b[0])
        charge_mat_elem = np.abs(get_mat_elem(flx_dict[key].charge_op(), state_a[1], state_b[1]))
        flx_data[key].append({"transition": transition[0] + transition[1], "freq": freq, "mat_elem": charge_mat_elem})

transm_data = []

for transition in transm_transitions:
    state_a = transm.state(transition[0])
    state_b = transm.state(transition[1])
    freq = np.abs(state_a[0] - state_b[0])
    charge_mat_elem = np.abs(get_mat_elem(transm.charge_op(), state_a[1], state_b[1]))
    transm_data.append({"transition": transition[0] + transition[1], "freq": freq, "mat_elem": charge_mat_elem})

In [5]:
save = False
if save:
    with open("flx_mat_elem_data.txt", 'w') as fp:
        json.dump(flx_data, fp)
    with open("transm_mat_elem_data.txt", 'w') as fp:
        json.dump(transm_data, fp)

In [6]:
phi = np.linspace(-3/2*np.pi, 3/2*np.pi, 1000)
levels = [0, 1, 2, 3]
wf_cr_xarray = flx_cr.wave_function(phi, levels=levels)
wf_cphase_xarray = flx_cphase.wave_function(phi, levels=levels)
wf_cr = {}
wf_cr["potential"] = list(np.array(wf_cr_xarray["potential"]))
wf_cr["energy"] = list(np.array(wf_cr_xarray["energy"]) )
wf_cr["wave_func"] = [list(np.array(wf_cr_xarray["wave_func"][x])) for x in range(4)]
wf_cr["phi"] = list(phi)
wf_cphase = {}
wf_cphase["potential"] = list(np.array(wf_cphase_xarray["potential"]))
wf_cphase["energy"] = list(np.array(wf_cphase_xarray["energy"]) )
wf_cphase["wave_func"] = [list(np.array(wf_cphase_xarray["wave_func"][x])) for x in range(4)]
wf_cphase["phi"] = list(phi)

In [7]:
save = True
if save:
    with open("wf_cr_data.txt", 'w') as fp:
        json.dump(wf_cr, fp)
    with open("wf_cphase_data.txt", 'w') as fp:
        json.dump(wf_cphase, fp)